In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Reshape
from pycaret.regression import *

In [3]:
df = pd.read_csv("data_augmented.csv")

In [4]:
df.head()

,dni,dhi,ghi,zenith,azimuth,optimal_tilt,max_gti,hour,air_temp,cloud_opacity,precipitable_water,precipitation_rate,relative_humidity,surface_pressure,Year,Month,Hour_sin,Hour_cos,Day_of_Year_sin,Day_of_Year_cos
0,0,0,0,2.967060,0.104720,0,0.0,1,12,17.8,8.8,0.0,78.7,975.9,2022,1,0.258819,0.965926,0.017213,0.999852
1,0,0,0,2.879793,-0.959931,0,0.0,2,12,32.7,8.7,0.0,81.5,975.7,2022,1,0.500000,0.866025,0.017213,0.999852
2,0,0,0,2.670354,-1.343904,0,0.0,3,11,32.5,8.8,0.0,82.3,975.6,2022,1,0.707107,0.707107,0.017213,0.999852
3,0,0,0,2.443461,-1.535890,0,0.0,4,11,2.4,8.8,0.0,82.3,975.6,2022,1,0.866025,0.500000,0.017213,0.999852
4,0,0,0,2.234021,-1.675516,0,0.0,5,10,2.1,8.9,0.0,83.0,975.5,2022,1,0.965926,0.258819,0.017213,0.999852


In [5]:
df.columns

Index(['dni', 'dhi', 'ghi', 'zenith', 'azimuth', 'optimal_tilt', 'max_gti',
       'hour', 'air_temp', 'cloud_opacity', 'precipitable_water',
       'precipitation_rate', 'relative_humidity', 'surface_pressure', 'Year',
       'Month', 'Hour_sin', 'Hour_cos', 'Day_of_Year_sin', 'Day_of_Year_cos'],
      dtype='object')

In [6]:
df.shape


(25968, 20)

In [7]:
# Définir la colonne cible (variable à prédire)
target = 'optimal_tilt'

In [8]:
# Initialiser PyCaret pour la régression
reg = setup(data=df,
            target=target,
            session_id=123,
            normalize_method='zscore',
            train_size=0.8,
            remove_outliers=True,
            use_gpu = True)


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3070 Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there a

,Description,Value
0,Session id,123
1,Target,optimal_tilt
2,Target type,Regression
3,Original data shape,"(25968, 20)"
4,Transformed data shape,"(24929, 20)"
5,Transformed train set shape,"(19735, 20)"
6,Transformed test set shape,"(5194, 20)"
7,Numeric features,19
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3070 Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there a

In [9]:
# Comparer les modèles disponibles
best_model = compare_models(
    sort='MSE'
)


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,15:04:47
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.4587,2.1630,1.4658,0.9954,0.1974,0.0547,0.8550
rf,Random Forest Regressor,0.5083,2.7580,1.6536,0.9941,0.1802,0.0645,1.5430
lightgbm,Light Gradient Boosting Machine,0.6585,2.8150,1.6712,0.9940,0.2175,0.0785,1.3180
xgboost,Extreme Gradient Boosting,0.6721,2.9132,1.7026,0.9937,0.2135,0.0781,0.8700
dt,Decision Tree Regressor,0.5598,6.8693,2.5944,0.9853,0.1836,0.0719,0.5910
gbr,Gradient Boosting Regressor,1.4355,10.6470,3.2503,0.9772,0.5021,0.1365,4.4400
knn,K Neighbors Regressor,1.7292,29.2028,5.3868,0.9375,0.5113,0.1805,0.5350
ada,AdaBoost Regressor,4.0618,39.5297,6.2827,0.9154,0.9217,0.4033,2.1580
ridge,Ridge Regression,7.3768,131.6747,11.4713,0.7182,1.5412,0.2823,0.4740
br,Bayesian Ridge,7.3775,131.6757,11.4714,0.7182,1.5413,0.2823,0.4860


In [10]:
# Afficher les détails du meilleur modèle
print(best_model)
baseline=best_model

ExtraTreesRegressor(n_jobs=-1, random_state=123)


In [11]:
# Ajuster le meilleur modèle sélectionné
tuned_model = tune_model(best_model)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,15:16:07
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.1997,9.1912,3.0317,0.9799,0.3928,0.1195
1,1.2625,7.3198,2.7055,0.9848,0.4033,0.1320
2,1.2354,10.3284,3.2138,0.9783,0.3889,0.1156
3,1.1643,5.2234,2.2855,0.9884,0.4116,0.1348
4,1.2136,7.3767,2.7160,0.9836,0.4160,0.1349
5,1.2968,9.0994,3.0165,0.9798,0.4253,0.1437
6,1.2320,10.6155,3.2581,0.9778,0.3663,0.1359
7,1.2544,9.1670,3.0277,0.9818,0.4154,0.1125
8,1.1801,7.0143,2.6485,0.9848,0.4056,0.1225


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [12]:
# Évaluer les performances du modèle
normal_results = predict_model(best_model)
tuned_results = predict_model(tuned_model)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.4429,1.6348,1.2786,0.9965,0.1690,0.0467


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.4429,1.6348,1.2786,0.9965,0.1690,0.0467


In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# 1️⃣ Charger les données
X = df.drop(columns=['optimal_tilt'])  # Variables d'entrée
y = df['optimal_tilt']  # Variable cible

# 2️⃣ Normalisation des données (important pour les réseaux neuronaux)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3️⃣ Séparation en train et test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 4️⃣ Construction du modèle Deep Learning (MLP)
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),  # Évite l'overfitting
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')  # Prédiction d'une valeur continue
])

# 5️⃣ Compilation du modèle
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='mse', metrics=['mae'])

# 6️⃣ Entraînement du modèle
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=100, batch_size=32, verbose=1)

# 7️⃣ Évaluation du modèle
loss, mae = model.evaluate(X_test, y_test)
print(f'MAE: {mae:.4f}, MSE: {loss:.4f}')

# 8️⃣ Prédictions
y_pred = model.predict(X_test)

# 9️⃣ Comparaison avec PyCaret
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse:.4f}, R²: {r2:.4f}')


Epoch 1/100
650/650 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 248.0299 - mae: 8.1047 - val_loss: 54.3705 - val_mae: 3.0044
Epoch 2/100
650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 54.0677 - mae: 3.0417 - val_loss: 35.5449 - val_mae: 2.0064
Epoch 3/100
650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 36.9750 - mae: 2.4524 - val_loss: 32.4780 - val_mae: 1.8509
Epoch 4/100
650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 28.9079 - mae: 2.2020 - val_loss: 28.1750 - val_mae: 1.6707
Epoch 5/100
650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 27.6016 - mae: 2.0862 - val_loss: 24.1717 - val_mae: 1.4791
Epoch 6/100
650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 23.7330 - mae: 1.9325 - val_loss: 21.2895 - val_mae: 1.3129
Epoch 7/100
650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 20.2301 - mae: 1.7804 - val_loss: 24.0377 - val_mae: 1.8963
Epoch 8/100
650/650 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 21.2351 - mae: 1.8518 - val_loss: 21.5844 - val_mae: 1.2709
Epoch 9/100
650/650 ━━━━━━━━━━━

In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 1️⃣ Charger les données
X = df.drop(columns=['optimal_tilt'])  # Variables d'entrée
y = df['optimal_tilt']  # Variable cible

# 2️⃣ Normalisation des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3️⃣ Reshape pour LSTM (LSTM attend un input de la forme [samples, timesteps, features])
X_lstm = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# 4️⃣ Séparation train/test
X_train, X_test, y_train, y_test = train_test_split(X_lstm, y, test_size=0.2, random_state=42)

# 5️⃣ Définition du modèle LSTM
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')  # Sortie pour une valeur continue
])

# 6️⃣ Compilation
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mse', metrics=['mae'])

# 7️⃣ Entraînement
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=50, batch_size=32, verbose=1)

# 8️⃣ Évaluation
loss, mae = model.evaluate(X_test, y_test)
print(f'MAE: {mae:.4f}, MSE: {loss:.4f}')

# 9️⃣ Prédictions et comparaison avec PyCaret
y_pred = model.predict(X_test)
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse:.4f}, R²: {r2:.4f}')


Epoch 1/50
650/650 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 270.8608 - mae: 7.6229 - val_loss: 44.6486 - val_mae: 2.1970
Epoch 2/50
650/650 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 33.1235 - mae: 2.1153 - val_loss: 30.3495 - val_mae: 1.7818
Epoch 3/50
650/650 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 28.2195 - mae: 1.8410 - val_loss: 25.1008 - val_mae: 1.4534
Epoch 4/50
650/650 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 21.6061 - mae: 1.6017 - val_loss: 25.5200 - val_mae: 1.3654
Epoch 5/50
650/650 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 20.4723 - mae: 1.4731 - val_loss: 21.1609 - val_mae: 1.2623
Epoch 6/50
650/650 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 16.9189 - mae: 1.3715 - val_loss: 23.2579 - val_mae: 1.4909
Epoch 7/50
650/650 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 16.1964 - mae: 1.3766 - val_loss: 20.8768 - val_mae: 1.2585
Epoch 8/50
650/650 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 15.8406 - mae: 1.3361 - val_loss: 18.1892 - val_mae: 1.1944
Epoch 9/50
650/650 ━━━━━━━━━━━━━━━━━━━━